In [1]:
import os
from typing import Annotated, Sequence, TypedDict
from langchain.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph.message import add_messages
from langchain_community.document_loaders import PyPDFLoader
from langgraph.checkpoint.memory import InMemorySaver  

c:\Users\aiear_vpvq6b8\Desktop\pro\hack\hask\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["AZURE_OPENAI_API_KEY"]=os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION")
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"]=os.getenv("AZURE_OPENAI_DEPLOYMENT")
from langchain.chat_models import init_chat_model

llm = init_chat_model(
    "azure_openai:gpt-4.1",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)

In [3]:
response=llm.invoke("hi how are you?")

In [4]:
token_usage = response.usage_metadata
print(f"Input tokens: {token_usage['input_tokens']}")
print(f"Output tokens: {token_usage['output_tokens']}")
print(f"Total tokens: {token_usage['total_tokens']}")


Input tokens: 12
Output tokens: 24
Total tokens: 36


In [5]:
EMBED = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [6]:
top_k=10

In [7]:
from langchain.tools import tool
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

def make_retriever_tool_from_text(file, name, desc):
    docs = PyPDFLoader(file).load()
    chunks = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=50
    ).split_documents(docs)
    
    vs = FAISS.from_documents(chunks, EMBED)
    retriever = vs.as_retriever(search_type="similarity",  # or "mmr" for maximal marginal relevance
        search_kwargs={"k": top_k})

    @tool  # Use @tool decorator without arguments
    def tool_func(query: str) -> str:
        """Retrieve documents based on query."""
        print(f"📚 Using tool: {name}")
        results = retriever.invoke(query)
        return "\n\n".join(doc.page_content for doc in results)
    
    # Set name and description after decoration
    tool_func.name = name
    tool_func.description = desc
    
    return tool_func

# Create your tools
# internal_tool_1 = make_retriever_tool_from_text(
#     "xxxx.pdf",
#     "xxxx_Docs",
#     "Search internal tech documents for proprietary architecture and methods."
# )

In [8]:
internal_tool_1=make_retriever_tool_from_text(
    file="TCS_Intern_Onboarding_Instructions.pdf",
    name="tcs_onboarding_agent",
    desc="This agent specializes in answering queries based solely on the 'TCS Intern Onboarding Instructions' PDF. It contains detailed guidance for new joiners at TCS, including Ultimatix activation, UxApps and Authenticator setup, mandatory profile updates (PAN, Aadhaar, address, bank details, photo), permanent access request creation, TCS email activation timelines, daily timesheet submission steps, mandatory iEvolve compliance courses, support contacts, and general onboarding workflows. The agent provides accurate, concise, step-by-step answers strictly from this PDF and rejects questions outside the document’s scope. Use this agent whenever a query is related to TCS onboarding, initial setup, compliance tasks, or process navigation."
)
internal_tool_1

StructuredTool(name='tcs_onboarding_agent', description="This agent specializes in answering queries based solely on the 'TCS Intern Onboarding Instructions' PDF. It contains detailed guidance for new joiners at TCS, including Ultimatix activation, UxApps and Authenticator setup, mandatory profile updates (PAN, Aadhaar, address, bank details, photo), permanent access request creation, TCS email activation timelines, daily timesheet submission steps, mandatory iEvolve compliance courses, support contacts, and general onboarding workflows. The agent provides accurate, concise, step-by-step answers strictly from this PDF and rejects questions outside the document’s scope. Use this agent whenever a query is related to TCS onboarding, initial setup, compliance tasks, or process navigation.", args_schema=<class 'langchain_core.utils.pydantic.tool_func'>, func=<function make_retriever_tool_from_text.<locals>.tool_func at 0x0000015359211940>)

In [9]:
tools=[internal_tool_1]

In [10]:
from langchain.agents import create_agent
agent = create_agent(
    llm,
    tools,
    system_prompt="You are the TCS Onboarding Instructions Agent. Your knowledge comes exclusively from the 'TCS Intern Onboarding Instructions' PDF. Your responsibility is to guide users through TCS onboarding steps with accuracy, clarity, and empathy. Always maintain a warm, reassuring, and supportive tone, understanding that new joiners may feel confused, stressed, or overwhelmed during onboarding.\n\nProvide answers that are:\n- Accurate and strictly based on the onboarding PDF.\n- Step-by-step, simple, and beginner-friendly.\n- Empathetic, calm, and encouraging.\n- Focused on reducing user confusion.\n\nWhen explaining any process (such as Ultimatix activation, UxApps setup, profile updates, access requests, timesheets, or iEvolve courses), acknowledge the user's feelings if they express difficulty, and respond with patience and clarity.\n\nIf a question is outside the PDF’s content, gently inform the user that the information is not included in the onboarding document, and guide them on what they *can* do next.\n\nYour role is to act as the supportive and knowledgeable agent for all TCS onboarding-related questions while keeping your responses empathetic, professional, and easy to follow.",
    checkpointer=InMemorySaver()
)

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import InMemorySaver
import uuid

# Initialize
# llm = init_chat_model("gpt-4o-mini")
checkpointer = InMemorySaver()

# Create agent
# agent = create_agent(
#     llm,
#     tools,
#     system_prompt="You are the TCS Onboarding Instructions Agent...",
#     checkpointer=checkpointer
# )

def chat_with_agent(thread_id: str = None):
    """Interactive chatbot loop with persistence."""
    if thread_id is None:
        thread_id = str(uuid.uuid4())
    
    config = {"configurable": {"thread_id": thread_id}}
    messages = []
    
    print("🎓 TCS Onboarding Agent")
    print("Type 'exit' to quit | 'new' for new conversation\n")
    
    while True:
        # Get user input
        user_input = input("You: ").strip()
        
        if user_input.lower() == "exit":
            print("Goodbye! 👋")
            break
        
        if user_input.lower() == "new":
            thread_id = str(uuid.uuid4())
            config = {"configurable": {"thread_id": thread_id}}
            messages = []
            print("\n🔄 New conversation started\n")
            continue
        
        if not user_input:
            continue
        
        # Add user message to history
        messages.append({"role": "user", "content": user_input})
        
        # Invoke agent with config
        try:
            result = agent.invoke(
                {"messages": messages},
                config
            )
            
            # Extract agent response
            assistant_message = result["messages"][-1].content
            print(f"\n🤖 Agent: {assistant_message}\n")
            
            # Update messages with agent response
            messages.append({"role": "assistant", "content": assistant_message})
            
        except Exception as e:
            print(f"\n❌ Error: {str(e)}\n")

# Run the chatbot
if __name__ == "__main__":
    # For single conversation
    chat_with_agent(thread_id="user_session_1")
    
    # OR for multiple independent sessions:
    # chat_with_agent()  # Creates unique thread_id

🎓 TCS Onboarding Agent
Type 'exit' to quit | 'new' for new conversation



In [ ]:
result=agent.invoke({
    "messages": [{"role": "user", "content": "how do i activate my ultimatix account"}],{"configurable": {"thread_id": "1"}})

📚 Using tool: tcs_onboarding_agent


In [38]:
print("\n✅ Final Answer:\n", result["messages"][-1].content)


✅ Final Answer:
 Activating your Ultimatix account is a very important step in your onboarding process. If you're feeling lost or overwhelmed, don't worry—I'll guide you through it step by step. Let’s get started!

---

### **BEFORE YOU BEGIN:**
Here’s what you need to have ready:
1. **Employee ID** - This is your unique identifier within TCS.
2. **Date of Birth (DDMMYYYY)** - To set your initial password.
3. **Access to a Mobile Device (Android/iOS)** - Required later for secure authentication.
4. **Internet Connection** - For accessing relevant portals.

---

### **3 Easy Steps to Activate Your Ultimatix Account:**

#### **1. Open the Login Page**  
- Go to the **Ultimatix website**: [https://www.ultimatix.net](https://www.ultimatix.net)

#### **2. Enter Your Login Credentials**
- **Username**: Enter your TCS Employee ID.  
- **Default Password**: Use the format `ChangePwd@DDMMYYYY`, where `DDMMYYYY` is your date of birth.  
  - **Example**: If your birthday is August 5, 2001, your 

In [39]:
result1=agent.invoke({
    "messages": [{"role": "user", "content": "what is default password for ultimatix?"}]})
print("\n✅ Final Answer:\n", result1["messages"][-1].content)

📚 Using tool: tcs_onboarding_agent

✅ Final Answer:
 The default password for logging into Ultimatix is based on your date of birth. It follows the format:

```
ChangePwd@DDMMYYYY
```

- Replace "DDMMYYYY" with your date of birth in the **Day, Month, Year** format.
  
For example:
- If your date of birth (DOB) is 5th August 2001, your default password will be:  
  `ChangePwd@05082001`

### Steps for First-Time Login:
1. Visit the **Ultimatix website** at [https://www.ultimatix.net](https://www.ultimatix.net).
2. Use your Employee ID as the **Username**.
3. Enter your default password (`ChangePwd@DDMMYYYY` as shown above).
4. On your first login, Ultimatix will prompt you to change this password. Follow the instructions to create a new, secure password of your choice.
  
If you feel stuck or face issues with your login, don’t worry—support is available. You can email: **Ultimatix.Login@tcs.com** for help.

Let me know if you need further details or encouragement—I’m here for you! 😊


In [40]:
result2=agent.invoke({
    "messages": [{"role": "user", "content": "how do i install UX APPs authenticator app?"}]})
print("\n✅ Final Answer:\n", result2["messages"][-1].content)

📚 Using tool: tcs_onboarding_agent

✅ Final Answer:
 It looks like the steps for installing the UxApps Authenticator app are detailed and straightforward. Let me guide you through this process step by step:

### **Step-by-step process to install and set up UxApps Authenticator:**

#### **Step 1: Verify Initial Requirements**
Before you start, ensure:
1. You have your **Employee ID**.
2. You know your **date of birth** in DDMMYYYY format.
3. You have access to a smartphone (Android/iOS) with an active internet connection.

---

#### **Step 2: Install the UxApps Application**
1. Open the browser on your mobile device.
2. Go to **[https://uxapps.ultimatix.net](https://uxapps.ultimatix.net)**.
3. Enter your **Employee ID** and tap on **Proceed**.
4. Select the **Password** option when prompted.
5. Enter your newly updated Ultimatix password.
6. Download the UxApps application from the link provided.
7. Install the application on your mobile phone.

---

#### **Step 3: Install and Set Up th

In [41]:
result3=agent.invoke({
    "messages": [{"role": "user", "content": "how do i install  authenticator app?"}]})
print("\n✅ Final Answer:\n", result3["messages"][-1].content)

📚 Using tool: tcs_onboarding_agent

✅ Final Answer:
 To install the Authenticator app during TCS onboarding, here is a step-by-step guide:

### Step 1: **Install UxApps**
1. On your mobile device, open a browser (like Chrome or Safari).
2. Go to **[https://uxapps.ultimatix.net](https://uxapps.ultimatix.net)**.
3. Enter your **Employee ID** and click "Proceed."
4. Select the "Password" option and enter your updated login password.
5. Download the **UxApps application** from the website.
6. Install the application on your smartphone.

---

### Step 2: **Access the Authenticator App via UxApps**
1. Open UxApps on your phone.
2. Log in using your updated password.
3. Once logged in, you will see a list of internal apps—find and select the **Authenticator App**.
4. Download and install the Authenticator on your phone.

---

### Step 3: **Set Up the Authenticator App**
1. Open the Authenticator app.
2. Log in using your updated password.
3. Set a **4-digit security PIN** within the app.
4. O

In [42]:
result4=agent.invoke({
    "messages": [{"role": "user", "content": "its been 24 hrs still no email access what to do"}]})
print("\n✅ Final Answer:\n", result4["messages"][-1].content)

📚 Using tool: tcs_onboarding_agent

✅ Final Answer:
 I'm sorry to hear that you're still unable to access your TCS email. I understand this situation can be frustrating, but don’t worry—let's try to resolve this together.

According to the onboarding instructions, your TCS email ID is typically created automatically after your Ultimatix account is activated, and it should be accessible within 24 hours. Since you’re already past this time frame, here’s what you can do next:

1. **Double-Check Ultimatix Activation:** Ensure that your Ultimatix account is activated. Without completion of this step, your email will not activate. Log in to [Ultimatix](https://www.ultimatix.net) using your credentials to confirm.

2. **Wait for 1-2 More Hours:** Occasionally, there might be a slight delay. Though 24 hours is the usual activation time, waiting an additional 1-2 hours can sometimes help if the system is slightly delayed.

3. **Contact Support for Assistance:**
   - Email: **Ultimatix.Login@tcs